In [2]:
import pandas
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names

2022-06-21 10:08:26.477678: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
DeepCTR version 0.9.1 detected. Your version is 0.9.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.1


In [3]:
from function_set import *

saved_model_path = '/data/linghui/saved_model/deepfm-ml-1m/'

saved_data_path = '/data/linghui/ml-1m/processed_data/'

test_data_path = saved_data_path + 'test_data_liuyi.csv'

#load data
test_data = pd.read_csv(test_data_path,index_col=0)

# model_name: DeepFM / 
# cate : all-feat / del-sf

model_params = {'model_name': 'DeepFM',
                'cate': 'liuyi-all-feats',
                'embedding_dim': 4,
                'batch_size': 256 }

data_params = {'dataset': 'ml-1m'}

check_path ,sparse_features = get_saved_model_details(model_params, data_params)

model = get_model(model_params,data_params)

test_input = {name: test_data[name].values for name in sparse_features}
label = test_data['rating'].values
y_true = label
y_pred = model.predict(test_input)

select_feats = ['user_id',"movie_id","rating","gender","age","occupation","zipcode"]

evaluate_data = test_data.loc[:,select_feats]
evaluate_data['predict'] = y_pred

2022-06-21 10:09:14.689860: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-06-21 10:09:14.709611: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-06-21 10:09:14.711167: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-06-21 10:09:14.711201: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-06-21 10:09:14.725302: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library l

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
age (InputLayer)                [(None, 1)]          0                                            
_______________________________________________________________________________________

2022-06-21 10:09:18.159347: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


In [16]:
female_test_true = female_test['rating'].to_numpy()
female_test_pred = female_test['predict'].to_numpy()
female_test_user = female_test['user_id'].to_numpy()

In [5]:
from sklearn.metrics import roc_auc_score
import numpy as np
from collections import defaultdict

In [6]:
evaluate_data

,user_id,movie_id,rating,gender,age,occupation,zipcode,predict
0,0,47,1,0,0,10,1588,0.939831
1,1,1737,1,1,6,16,2248,0.823337
2,2,1900,1,1,2,15,1863,0.927760
3,3,1774,1,1,4,7,140,0.912610
4,4,279,1,1,2,20,1938,0.555670
...,...,...,...,...,...,...,...,...
301995,6039,2993,0,1,2,6,466,0.171150
301996,6039,2166,0,1,2,6,466,0.419440
301997,6039,286,0,1,2,6,466,0.032339
301998,6039,70,0,1,2,6,466,0.333478


In [7]:
class rec_metrics(object):
    def __init__(self, evaluate_data,k):
        self.users = np.array(evaluate_data['user_id'].to_list())
        self.y_true = np.array(evaluate_data['rating'].to_list())
        self.y_pred = np.array(evaluate_data['predict'].to_list())
        self.item_id = np.array(evaluate_data['movie_id'].to_list())
        self.item_cate = np.array(evaluate_data['genres'].to_list())
        self.item_pop = np.array(evaluate_data['popularity'].to_list())
        self.k = k
        self.select_cols = [self.y_true,self.y_pred,self.item_id,self.item_cate,self.item_pop]
    def get_user_pred(self):
        """
        divide the result into different group by user id

        Args:
        y_true: array, all true labels of the data
        y_pred: array, the predicted score
        users: array, user id

        Return:
        user_pred: dict, key is user id and value is the labels and scores of each user
        """
        user_pred = {}
        for i, u in enumerate(self.users):
            if u not in user_pred:
                # user_pred[u] = [[self.y_true[i]], [self.y_pred[i]],[self.item_id[i]],[self.item_cate[i]]]
                user_pred[u] = [[feat[i]] for feat in self.select_cols]
            else:
                for index in range(len(self.select_cols)):
                    user_pred[u][index].append(self.select_cols[index][i])
                # user_pred[u][0].append(self.y_true[i])
                # user_pred[u][1].append(self.y_pred[i])
                # user_pred[u][2].append(self.item_id[i])
                # user_pred[u][3].append(self.item_cate[i])
        return user_pred
    def get_user_topk(self):
        
        """
        sort y_pred and find topk results
        this function is used to find topk predicted scores
        and the corresponding index is applied to find the corresponding labels

        """
        user_pred = self.get_user_pred()
        for u in user_pred:
            idx = np.argsort(user_pred[u][1])[::-1][:self.k]
            for i in range(len(self.select_cols)):
                user_pred[u][i] = np.array(user_pred[u][i])[idx]
        return user_pred

    def auc_score(self):
        return roc_auc_score(self.y_true, self.y_pred)
    
    def gauc_score(self,weights=None):
        """
        Args:
        y_true: array, dim(N, ), all true labels of the data
        y_pred: array, dim(N, ), the predicted score
        users: array, dim(N, ), user id
        weight: dict, it contains weights for each group.
            if it is None, the weight is equal to the number
            of times the user is recommended
        Return:
        score: float, GAUC
        """
        assert len(self.y_true) == len(self.y_pred) and len(self.y_true) == len(self.users)

        user_pred = self.get_user_topk()
        score = 0
        num = 0
        for u in user_pred.keys():
            auc = auc_score(user_pred[u][0], user_pred[u][1])
            if weights is None:
                wg = len(user_pred[u][0])
            else:
                wg = weights[u]
            auc *= wg
            num += wg
            score += auc
        return score / num
    
    def log_loss(self):
        score = self.y_true * np.log(self.y_pred) + (1 - self.y_true) * np.log(1 - self.y_pred)
        return -score.sum() / len(self.y_true)
    
    def ndcg_score(self,user_pred):
        """compute NDCG
        Args:
        user_pred: dict, computed by get_user_topk()
        """
        rank = np.arange(1, self.k+1, 1)
        idcgs = 1. / np.log2(rank + 1)
        idcg = sum(idcgs)
        score = 0
        for u in user_pred:
            dcgs = idcgs[np.where(user_pred[u][0] == 1)]
            dcg = sum(dcgs)
            score += dcg / idcg
        return score / len(user_pred.keys())
    
    def cate_diversity_score(self,user_pred):
        score = 0
        for u in user_pred:
            item_list = user_pred[u][3]
            item_list = sum(item_list,[])
            cate = set(item_list)
            cate_num = len(cate)
            score += cate_num
        return score / len(user_pred.keys())
    
    def popularity_score(self,user_pred):
        score = 0
        for u in user_pred:
            pop_list = user_pred[u][4]
            pop_score = np.mean(pop_list)
            score += pop_score
        return score / len(user_pred.keys())
    
    def hit_score(self,user_pred):
        score = 0
        for u in user_pred:
            if 1 in user_pred[u][0]:
                score += 1.0
        return score / len(user_pred.keys())
    def mrr_score(self,user_pred):
        score = 0
        for u in user_pred:
            if 1 in user_pred[u][0]:
                score += 1.0 / (np.where(user_pred[u][0] == 1)[0][0] + 1)
        return score / len(user_pred.keys())
    def recall_score(self,user_pred):
        score = 0
        for u in user_pred:
            score += sum(user_pred[u][0]) * 1. / len(user_pred[u][0])
        return score / len(user_pred.keys())
    
    def get_all_metrics(self):
        user_pred = self.get_user_topk()
        
        log_loss = self.log_loss()
        auc_score = self.auc_score()
        # gauc_score = self.gauc_score()
        ndcg_score = self.ndcg_score(user_pred)
        hit_score = self.hit_score(user_pred)
        mrr_score = self.mrr_score(user_pred)
        recall_score = self.recall_score(user_pred)
        cate_diversity_score = self.cate_diversity_score(user_pred)
        pop_score = self.popularity_score(user_pred)
        
        result = pd.DataFrame()
        result['log_loss'] = [log_loss]
        result['auc'] = [auc_score]
        result['ndcg'] = [ndcg_score]
        result['hit'] = [hit_score]
        result['mrr'] = [mrr_score]
        result['cate_diversity'] = [cate_diversity_score]
        result['popularity'] = [pop_score]
        return result


In [10]:
class rec_fairness_metrics(object):
    def __init__(self, group_target, group_compare):
        self.group_target = group_target
        self.group_compare = group_compare
        self.label_column = 'rating'
        self.pred_column = 'predict'
        
        self.group_target_metrics = rec_metrics(group_target,5)
        self.group_compare_metrics = rec_metrics(group_compare,5)
    
    def sub_pred_true(self,data):
        y_true = np.array(data[self.label_column].to_list())
        y_pred = np.array(data[self.pred_column].to_list())
        result = np.mean(y_pred - y_true)
        return result
        
    def value_unfairness(self,select_group,compare_group):
        item_id = select_group['movie_id'].unique()
        val_unfairness_list = []
        for i in item_id:
            user_select_group = select_group.loc[select_group['movie_id'] == i]
            if len(user_select_group) == 0:
                val_target = 0
            else:
                val_target = self.sub_pred_true(user_select_group)
            user_compare_group = compare_group.loc[compare_group['movie_id'] == i]
            if len(user_compare_group) == 0:
                val_compare = 0
            else:
                val_compare = self.sub_pred_true(user_compare_group)
            val_unfairness_list.append(val_target - val_compare)
        return(np.mean(val_unfairness_list))
    
    def parity_unfairness(self,select_group,compare_group):
        item_id = select_group['movie_id'].unique()
        for i in item_id:
            user_select_group = select_group.loc[select_group['movie_id'] == i]
            if len(user_select_group) == 0:
                par_target = 0
            else:
                par_target = np.mean(user_select_group[self.pred_column].to_list())
            user_compare_group = compare_group.loc[compare_group['movie_id'] == i]
            if len(user_compare_group) == 0:
                par_compare = 0
            else:
                par_compare = np.mean(user_compare_group[self.pred_column].to_list())
            parity_unfairness = par_target - par_compare
        return parity_unfairness
    
    def metric_unfairness(self):
        metrics_target = self.group_target_metrics.get_all_metrics()
        metrics_compare = self.group_compare_metrics.get_all_metrics()
        metrics_unfairness = metrics_target.to_numpy() - metrics_compare.to_numpy()
        result_unfairness = pd.DataFrame(metrics_unfairness,columns=metrics_target.columns)
        return result_unfairness
    
    def get_all_unfairness_metrics(self):
        result_unfairness = self.metric_unfairness()
        
        select_group = self.group_target
        compare_group = self.group_compare
        value_unfairness = self.value_unfairness(select_group,compare_group)
        parity_unfairness = self.parity_unfairness(select_group,compare_group)
        
        result_unfairness['value'] = value_unfairness
        result_unfairness['parity'] = parity_unfairness
        return result_unfairness
        

In [12]:
# load movie data
# data path
data_path = '/data/linghui/ml-1m/'
# data_path = '/home/nesa320/huizhong/movielens/data-ml/'
# file : ratings.dat  movies.dat  users.dat

user_file = data_path + 'users.dat'
rating_file = data_path + 'ratings.dat'
movie_file = data_path + 'movies.dat'

movie_data = pd.read_csv(movie_file, sep='::', header=None, error_bad_lines=False,engine='python', encoding="ISO-8859-1")
movie_data.columns = ['movie_id', 'title', 'genres']
lbe_feats = ['movie_id']
for feat in lbe_feats:
    lbe = LabelEncoder()
    movie_data[feat] = lbe.fit_transform(movie_data[feat])
movie_data

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,movie_id,title,genres
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3878,Meet the Parents (2000),Comedy
3879,3879,Requiem for a Dream (2000),Drama
3880,3880,Tigerland (2000),Drama
3881,3881,Two Family House (2000),Drama


In [13]:
def split(x):
    key_ans = x.split('|')
    for key in key_ans:
        if key not in key2index:
            # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

key2index = {}
genres_list = list(map(split, movie_data['genres'].values))
# genres_length = np.array(list(map(len, genres_list)))
# max_len = max(genres_length)
# # Notice : padding=`post`
# genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post')


In [15]:
movie_data['genres'] = genres_list
movie_data = movie_data.drop(columns=['title'])
movie_data

,movie_id,genres
0,0,"[1, 2, 3]"
1,1,"[4, 2, 5]"
2,2,"[3, 6]"
3,3,"[3, 7]"
4,4,[3]
...,...,...
3878,3878,[3]
3879,3879,[7]
3880,3880,[7]
3881,3881,[7]


In [16]:

def create_ml_1m_data(data):
    data = data.sort_values(['user_id','timestamp'])
    num = len(data.user_id.value_counts())
    test_pd = pd.DataFrame(columns=data.columns)
    for i in range(num):
        user_select_group = data.loc[data['user_id'] == i]
        user_pd = user_select_group.iloc[-1]
        test_pd = test_pd.append(user_pd)
    drop_index = test_pd.index.to_list()
    train_pd = data[~data.index.isin(drop_index)]
    return train_pd,test_pd

#load rating data

rating_data = pd.read_csv(rating_file, sep='::', header=None,error_bad_lines=False,engine='python')
rating_data.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
# rating_data = rating_data.dropna()
# # rating_data['movie_id'] = rating_data['movie_id'].apply(pd.to_numeric,errors='coerce')

# rating_data['movie_id'] = rating_data['movie_id'].astype('int') 
# rating_data['timestamp'] = rating_data['timestamp'].round(0).astype('int') 
lbe_feats = ['user_id', 'movie_id']
for feat in lbe_feats:
    lbe = LabelEncoder()
    rating_data[feat] = lbe.fit_transform(rating_data[feat])

rating_data

train_set , test_set = create_ml_1m_data(rating_data)

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
train_set

,user_id,movie_id,rating,timestamp
31,0,2969,4,978300019
22,0,1178,5,978300055
27,0,1574,4,978300055
37,0,957,5,978300055
24,0,2147,3,978300103
...,...,...,...,...
999923,6039,225,5,997454398
1000019,6039,2709,4,997454429
999988,6039,1741,4,997454464
1000172,6039,1618,3,997454464


In [18]:
popularity_list = []

for i in movie_data['movie_id'].to_list():
    select_movie = train_set.loc[train_set['movie_id'] == i]
    popularity_list.append(len(select_movie) / 994169 * 10000)

In [19]:
movie_data['popularity'] = popularity_list
# movie_data.sort_values(['popularity'])

In [20]:
# female = pd.merge(female,movie_data,on=['movie_id'],how='left')
# female
evaluate_data = pd.merge(evaluate_data,movie_data,on=['movie_id'],how='left')
evaluate_data

,user_id,movie_id,rating,gender,age,occupation,zipcode,predict,genres,popularity
0,0,47,1,0,0,10,1588,0.939831,"[1, 2, 15, 6]",3.812229
1,1,1737,1,1,6,16,2248,0.823337,[13],11.134928
2,2,1900,1,1,2,15,1863,0.927760,[11],10.360412
3,3,1774,1,1,4,7,140,0.912610,"[2, 3]",11.326042
4,4,279,1,1,2,20,1938,0.555670,[7],7.010880
...,...,...,...,...,...,...,...,...,...,...
301995,6039,2993,0,1,2,6,466,0.171150,"[8, 7, 14]",2.323549
301996,6039,2166,0,1,2,6,466,0.419440,"[7, 14]",7.051115
301997,6039,286,0,1,2,6,466,0.032339,"[3, 6]",0.291701
301998,6039,70,0,1,2,6,466,0.333478,[8],0.915337


In [21]:
female = evaluate_data.loc[evaluate_data['gender'] == 0]
male = evaluate_data.loc[evaluate_data['gender'] == 1]
# female_test = female.loc[female['user_id'].isin([0,5])]
# # male_test = male.loc[male['movie_id'] == 0].head(2)
# female_test

In [291]:
test_cate = rec_metrics(female_test,5)
user_pred = test_cate.get_user_topk()
user_pred

{0: [array([0, 1, 0, 0, 0]),
  array([0.97253954, 0.9398309 , 0.9091568 , 0.87913299, 0.765513  ]),
  array([  33,   47, 1294, 1371, 1832]),
  array([list([2, 3, 7]), list([1, 2, 15, 6]), list([6]), list([3, 7]),
         list([13])], dtype=object),
  array([17.47187852,  3.81222911, 13.49871098,  6.68900358, 11.38639406])],
 5: [array([1, 0, 0, 0, 0]),
  array([0.93760329, 0.91943955, 0.90877426, 0.90066606, 0.88169479]),
  array([ 583,  370, 1878, 1943, 1904]),
  array([list([3, 6, 10]), list([2, 3]), list([15, 6]), list([3, 12, 18]),
         list([11])], dtype=object),
  array([10.49117404, 14.01170224,  2.67560143,  7.17181888,  5.68313838])]}

In [313]:
sample_num = 2000
l = female.user_id.unique()
fe_sample = np.random.choice(l,size=sample_num)
fe_sample = female.loc[female['user_id'].isin(fe_sample)]

l1 = male.user_id.unique()
ma_sample = np.random.choice(l1,size=sample_num)
ma_sample = male.loc[male['user_id'].isin(ma_sample)]
test = rec_fairness_metrics(fe_sample,ma_sample)
test.get_all_unfairness_metrics()

,log_loss,auc,ndcg,hit,mrr,cate_diversity,popularity,value,parity
0,0.022107,0.001764,0.00318,0.008142,0.009634,-0.04798,-0.247762,0.009813,-0.022274


In [26]:
# all test - female -  male

test = rec_fairness_metrics(female,male)
a = test.get_all_unfairness_metrics()

In [29]:
sensitive_features = ['gender' , 'age']

feat_value = evaluate_data['age'].value_counts()
feat_value = feat_value.index
feat_value

Int64Index([2, 3, 1, 4, 5, 6, 0], dtype='int64')

In [ ]:
def creat_sparse_rule_query(rule):
    result = ''
    key = rule.keys()
    length = len(key)
    count = 0
    for i in key:
        count = count + 1
        result = result + '( ' + i +' == ' + str(rule[i]) + ')'
        # elif i[0] == 'I':
        #     result = result + '(' + str(rule[i][0]) + ' <= ' + i + ' <= ' +  str(rule[i][1]) + ')'
        if count < length:
            result = result + ' & '
    return result
